In [97]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

print('blah')

blah


In [98]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [99]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_softmax_stack = nn.Sequential(
            nn.Linear(8, 6),
            nn.LogSoftmax(),
            nn.Linear(6, 4),
            nn.LogSoftmax(),
            nn.Linear(4, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_softmax_stack(x)
        return logits

In [101]:
model = NeuralNetwork().to(device)
model_path = 'my_model.pt'

if(os.path.exists(model_path)):
    model.load_state_dict(torch.load(model_path))
    print("Loaded saved state")

print(model)

Loaded saved state
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_softmax_stack): Sequential(
    (0): Linear(in_features=8, out_features=6, bias=True)
    (1): LogSoftmax(dim=None)
    (2): Linear(in_features=6, out_features=4, bias=True)
    (3): LogSoftmax(dim=None)
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [104]:
X = torch.rand(1, 8, device=device)
logits = model(X)
print(f"All returns: {logits}")

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

All returns: tensor([[-0.4271]], device='mps:0', grad_fn=<LinearBackward0>)
Predicted class: tensor([0], device='mps:0')


In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

play_data = pd.read_table('TrainingQueue.tsv')


X = play_data.drop(columns=['MyPaddleVelocity','Unnamed: 9']).dropna()
y = play_data['MyPaddleVelocity'].dropna() 



In [108]:
class PongDataset(Dataset):
    def __init__(self, X, y):
        self.features = X.values
        self.labels = y.values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label

play_data = pd.read_table('TrainingQueue.tsv')
X = play_data.drop(columns=['MyPaddleVelocity', 'Unnamed: 9']).dropna()
y = play_data['MyPaddleVelocity'].dropna()

print (X)

       MyPaddlePosition  MyPaddleTimeAsIs  OpponentPaddle  OpponentVelocity  \
0                -1.000          1.384546        -2.00000               0.0   
1                -1.000          1.414189        -2.00000               0.0   
2                -1.000          1.439879        -2.00000               0.0   
3                -1.000          1.462690        -2.00000               0.0   
4                -1.000          1.487308        -2.00000               0.0   
...                 ...               ...             ...               ...   
15496             0.245          2.207550         1.70413               0.0   
15497             0.245          2.227112         1.70413               0.0   
15498             0.245          2.246399         1.70413               0.0   
15499             0.245          2.267944         1.70413               0.0   
15500             0.245          2.289093         1.70413               0.0   

       BallPositionX  BallPositionY  BallVelocityX 

In [109]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create data loaders for the training and validation sets
train_dataset = PongDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = PongDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [110]:
# Define the loss function and optimizer
loss_fn = nn.MSELoss()
learning_rate = .001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model for the specified number of epochs
num_epochs = 30
for epoch in range(num_epochs):
    for batch in train_loader:
        # Move the batch of training data to the device
        X_batch, y_batch = batch
        X_batch = X_batch.float()
        y_batch = y_batch.float()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch.unsqueeze(1))
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Evaluate the model on the validation set
    with torch.no_grad():
        for batch in val_loader:
            # Move the batch of validation data to the device
            X_batch, y_batch = batch
            X_batch = X_batch.float()
            y_batch = y_batch.float()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            y_pred = model(X_batch)
            val_loss = loss_fn(y_pred, y_batch.unsqueeze(1))
            
        # Print progress
        print(f"Epoch [{epoch}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")



/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [0/30], Train Loss: 0.5842, Val Loss: 4.3005
Epoch [1/30], Train Loss: 1.6620, Val Loss: 4.2108
Epoch [2/30], Train Loss: 0.2995, Val Loss: 3.7702
Epoch [3/30], Train Loss: 2.5758, Val Loss: 3.0740
Epoch [4/30], Train Loss: 0.2742, Val Loss: 2.9900
Epoch [5/30], Train Loss: 1.5837, Val Loss: 2.8679
Epoch [6/30], Train Loss: 1.1724, Val Loss: 2.8375
Epoch [7/30], Train Loss: 0.9759, Val Loss: 2.8634
Epoch [8/30], Train Loss: 1.2868, Val Loss: 2.8619
Epoch [9/30], Train Loss: 2.4291, Val Loss: 2.9601
Epoch [10/30], Train Loss: 1.8179, Val Loss: 2.9209
Epoch [11/30], Train Loss: 0.2827, Val Loss: 2.9678
Epoch [12/30], Train Loss: 2.3782, Val Loss: 2.9691
Epoch [13/30], Train Loss: 1.7127, Val Loss: 2.9275
Epoch [14/30], Train Loss: 0.5883, Val Loss: 2.8750
Epoch [15/30], Train Loss: 1.4983, Val Loss: 2.9776
Epoch [16/30], Train Loss: 1.6749, Val Loss: 2.9373
Epoch [17/30], Train Loss: 3.0150, Val Loss: 2.8921
Epoch [18/30], Train Loss: 1.3110, Val Loss: 2.9116
Epoch [19/30], Train L

In [111]:
# Evaluate the trained model on the test set
model.eval()

test_loss = 0
correct = 0
with torch.no_grad():
    for batch in val_loader:
        # Move the batch of testing data to the device
        X_batch, y_batch = batch
        X_batch = X_batch.float()
        y_batch = y_batch.float()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Forward pass
        y_pred = model(X_batch)

        # Compute loss
        test_loss += loss_fn(y_pred, y_batch).item()

        # Compute number of correct predictions
        correct += (y_pred.squeeze().round() == y_batch.round()).type(torch.float).sum().item()


test_loss /= len(val_loader.dataset)
accuracy = correct / len(val_loader.dataset)

print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2%}')


/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([29])) that is different to the input size (torch.Size([29, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test set: Average loss: 0.1138, Accuracy: 66.17%


In [112]:
# Export model
model.to('cpu')
onnx_file_name = "torch_model.onnx"
dummy_input = torch.randn(1,8)

# Export the trained model to ONNX format
torch.onnx.export(model,                 # model being run
                  dummy_input,           # dummy input tensor
                  onnx_file_name,        # output file name
                  verbose=True,          # print out a lot of information
                  input_names=["input"], # specify the name of the input
                  output_names=["output"], # specify the name of the output
                  opset_version=10)      # the ONNX version to use


torch.save(model.state_dict(), model_path)

Exported graph: graph(%input : Float(1, 8, strides=[8, 1], requires_grad=0, device=cpu),
      %linear_softmax_stack.0.weight : Float(6, 8, strides=[8, 1], requires_grad=1, device=cpu),
      %linear_softmax_stack.0.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %linear_softmax_stack.2.weight : Float(4, 6, strides=[6, 1], requires_grad=1, device=cpu),
      %linear_softmax_stack.2.bias : Float(4, strides=[1], requires_grad=1, device=cpu),
      %linear_softmax_stack.4.weight : Float(1, 4, strides=[4, 1], requires_grad=1, device=cpu),
      %linear_softmax_stack.4.bias : Float(1, strides=[1], requires_grad=1, device=cpu)):
  %/flatten/Flatten_output_0 : Float(1, 8, strides=[8, 1], requires_grad=0, device=cpu) = onnx::Flatten[axis=1, onnx_name="/flatten/Flatten"](%input), scope: __main__.NeuralNetwork::/torch.nn.modules.flatten.Flatten::flatten # /Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/flatten.py:46:0
  %/linear_softmax_stack/linear_softm

/Users/logan/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1194: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return forward_call(*input, **kwargs)
